# Day 004 
## EDA 之資料類型介紹/EDA: 欄位的資料類型介紹及處理（數值或連續）
### 常見的欄位資料類型有:
1. float64: 浮點數，用來表示離散(discrete)或連續(continuous)變數。
2. int64: 整數，用來表示離散(discrete)或連續(continuous)變數。
3. object: 包含字串，用來表示類別型變數。


### 在初步 EDA 的過程，我們無可避免會想問的問題如
- 不同資料類型各有多少個欄位？
- 類別型欄位(pandas 中的 object type)的類別數量?
- 模型怎麼處理類別型的資料？有什麼表示方法？

### 1.不同資料類型各有多少個欄位?

這裡我們會使用到兩個method，一個屬於DataFrame，另一個則作用在Series上。
1. [dtypes](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dtypes.html): <br>
    Return一個Series，其中的元素為每個column的資料類型，且index為DataFrame的欄名。
2. [value_counts](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html): <br>
    Returns object containing counts of unique values.

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
# 設定 data_path, 並讀取 app_train和app_test
dir_data = '../Data/Part01'
f_app_train = os.path.join(dir_data, 'application_train.csv')
app_train = pd.read_csv(f_app_train)

f_app_test = os.path.join(dir_data, 'application_test.csv')
app_test = pd.read_csv(f_app_test)

In [3]:
# 檢視資料中各個欄位類型的數量
app_train.dtypes.value_counts()

float64    65
int64      41
object     16
dtype: int64

### 2.類別型(Categorical)欄位的類別數量?
這裡使用的method有
1. [select_dtypes](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.select_dtypes.html) <br>
    作用於DataFrame，依照attribute include、exclude來決定回傳的subset(依然是DataFrame)。
2. [apply](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html) <br>
    把一行、列視為一個基本單位，對其施加一個函式，DataFrame的axis='column' 與 axis = 1 為對行(row)做運算，因為他是指alone the column，而axis='index'則為對列(column)運算，所以與axis = 0相同。
3. [pd.Series.nunique](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.nunique.html) <br>
    Return一個object中unique元素的數量。

In [4]:
app_train.select_dtypes(include=["object"]).apply(pd.Series.nunique, axis = 0)

NAME_CONTRACT_TYPE             2
CODE_GENDER                    3
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
NAME_TYPE_SUITE                7
NAME_INCOME_TYPE               8
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
OCCUPATION_TYPE               18
WEEKDAY_APPR_PROCESS_START     7
ORGANIZATION_TYPE             58
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
WALLSMATERIAL_MODE             7
EMERGENCYSTATE_MODE            2
dtype: int64

### 3.模型怎麼處理類別型的資料？有什麼表示方法？
簡單來說我們有兩種方法來處理類別型資料
- Label encoding: 把每個類別 mapping 到某個整數，不會增加新欄位
- One Hot encoding: 為每個類別新增一個欄位，用 0/1 表示是否


## HW1

請詳細閱讀 [Label Encoder vs. One Hot Encoder in Machine Learning](https://medium.com/@contactsunny/label-encoder-vs-one-hot-encoder-in-machine-learning-3fc273365621)

這兩個編碼器來自SciKit Learn library，作用為把categorical或text data轉為預測模型可以理解的數字。
1. Label Encoder: <br>
   `sklearn.preprocessing.LabelEncoder`的[document](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) <br>
   Label Encoder的工作就是把要轉換的欄在每個欄位原本的位置上轉成數字，程式碼如下
   ```python
    from sklearn.preprocessing import LabelEncoder
    labelencoder = LabelEncoder()
    x[:, 0] = labelencoder.fit_transform(x[:, 0])
   ```
   但這樣的作法會造成一個問題，因為同一欄有不同的數字，雖然它們本來是毫無相關的categorical data，但模型會誤以為它們有關係(如大小關係)，因此我們需要One Hot Encoder。
   
   
2. One Hot Encoder: <br>
    `sklearn.preprocessing.OneHotEncoder`的[document](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) <br>
   One Hot Encoder的工作就是把要轉換的欄中每個欄位獨立出來成為新的一欄，並用0、1代表此列資料是否屬於該欄位。
   ```python
    from sklearn.preprocessing import OneHotEncoder
    onehotencoder = OneHotEncoder(categorical_features = [0])
    x = onehotencoder.fit_transform(x).toarray()
   ```


### 以下就來示範兩種方式的應用
### Label encoding
由HW1可以發現，Label encoding 的表示方式會讓同一個欄位底下的類別之間有大小關係 (0<1<2<...)，所以在這裡我們只對有類別數量小於等於 2 的類別型欄位示範使用 Label encoding，但不表示這樣處理是最好的，一切取決於欄位本身的意義適合哪一種表示方法

In [5]:
# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


## HW2
將下列部分資料片段 sub_train 使用 One Hot encoding, 並觀察轉換前後的欄位數量 (使用 shape) 與欄位名稱 (使用 head) 變化

In [6]:
sub_train = pd.DataFrame(app_train['WEEKDAY_APPR_PROCESS_START'])
print(sub_train.shape)
sub_train.head()

(307511, 1)


,WEEKDAY_APPR_PROCESS_START
0,WEDNESDAY
1,MONDAY
2,MONDAY
3,WEDNESDAY
4,THURSDAY


### One Hot encoding
這裡使用的是較簡單的方式，利用[get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)來實現One Hot encoding

In [7]:
sub_train = pd.get_dummies(sub_train)
print(sub_train.shape)
sub_train.head()

(307511, 7)


,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY
0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0
3,0,0,0,0,0,0,1
4,0,0,0,0,1,0,0


> 可以觀察到row的數量不變，但column變成7（因為一星期有七天），且原來的類別型欄位都轉為 0/1 了。

## 課後閱讀
[pandas 數據類型](https://blog.csdn.net/claroja/article/details/72622375.)